# Riiid! Answer Correctness Prediction
## Introduction
In this competition you will predict which questions each student is able to answer correctly. You will loop through a series of batches of questions. Once you make that prediction, you can move on to the next batch.

This competition is different from most Kaggle Competitions in that:
* You can only submit from Kaggle Notebooks
* You must use our custom **`riiideducation`** Python module.  The purpose of this module is to control the flow of information to ensure that you are not using future data to make predictions.  If you do not use this module properly, your code may fail.

## In this Starter Notebook, we'll show how to use the **`riiideducation`** module to get the test features and make predictions.
## TL;DR: End-to-End Usage Example
```
import riiideducation
env = riiideducation.make_env()

# Training data is in the competition dataset as usual
train_df = pd.read_csv('/kaggle/input/riiideducation/train.csv', low_memory=False)
train_my_model(train_df)
iter_test = env.iter_test()
for (test_df, sample_prediction_df) in iter_test:
    test_df['answered_correctly'] = 0.5
    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])```
Note that `train_my_model` is a function you need to write for the above example to work.

## In-depth Introduction
First let's import the module and create an environment.

In [ ]:
%%time
# Code you have previously used to load data
import time
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
#from sklearn.pipeline import make_pipeline
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.model_selection import cross_val_score
#from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingRegressor
#from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
#from sklearn.tree import DecisionTreeRegressor
from learntools.core import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)


In [ ]:
import riiideducation
import pandas as pd

# You can only call make_env() once, so don't lose it!
env = riiideducation.make_env()

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

### Training data is in the competition dataset as usual
It's larger than will fit in memory with default settings, so we'll specify more efficient datatypes and only load a subset of the data for now.

In [ ]:
example = pd.read_csv ('/kaggle/input/riiid-test-answer-prediction/example_sample_submission.csv')
example

In [ ]:
train_df = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/train.csv', low_memory=False, nrows=10**5, 
                       dtype={'row_id': 'int64', 'timestamp': 'int64', 'user_id': 'int32', 'content_id': 'int16', 'content_type_id': 'int8',
                              'task_container_id': 'int16', 'user_answer': 'int8', 'answered_correctly': 'int8', 'prior_question_elapsed_time': 'float32', 
                             'prior_question_had_explanation': 'boolean',
                             }
                      )
train_df.head()

In [ ]:
train_df._get_numeric_data().corr()

In [ ]:
train_df.dropna(inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder
lbl = LabelEncoder()
lbl.fit(list(train_df[['prior_question_had_explanation']].values.ravel()))
train_df['prior_question_had_explanation'] = lbl.transform(list(train_df[['prior_question_had_explanation']].values.ravel()))

In [ ]:
#X = np.asarray(train_df[['content_id', 'prior_question_had_explanation']])
train_X = train_df[['content_id', 'prior_question_had_explanation']]
train_X.head()

In [ ]:
#Y = np.asarray(train_df[['answered_correctly']])
train_Y = train_df[['answered_correctly']]
train_Y.dtypes

In [ ]:
#train_X, val_X, train_y, val_y = train_test_split(X, Y, random_state=1)

In [ ]:
# Rough Model to Start With       

iowa_model = GradientBoostingRegressor(random_state=0,max_depth=3,max_leaf_nodes=None,n_estimators=100,
                                       learning_rate=0.08,min_samples_leaf=1,
                                       )
                                           
# Fit Model

iowa_model.fit(train_X, train_Y.values.ravel())

# Make validation predictions and calculate mean absolute error

val_predictions = iowa_model.predict(train_X)
#val_mae = mean_absolute_error(val_predictions, train_Y)
#print("Validation MAE when not specifying max_leaf_nodes: {:,.0f}".format(val_mae))

In [ ]:
val_predictions.dtype

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(train_Y, val_predictions)

## `iter_test` function

Generator which loops through each batch of questions in the test set. You have direct access to the example test rows for your convenience, but your code will only be able to get rows from the real test set via the API. Once you call **`predict`** you can continue on to the next batch.

Yields:
* While there are more batch(es) and `predict` was called successfully since the last yield, yields a tuple of:
    * `test_df`: DataFrame with the test features for the next batch, and user responses for the previous batch.
    * `sample_prediction_df`: DataFrame with an example prediction.  Intended to be filled in and passed back to the `predict` function.
* If `predict` has not been called successfully since the last yield, prints an error and yields `None`.

In [ ]:
# You can only iterate through a result from `env.iter_test()` once
# so be careful not to lose it once you start iterating.
iter_test = env.iter_test()

Let's get the data for the first test batch and check it out.

In [ ]:
(test_df, sample_prediction_df) = next(iter_test)
test_df

In [ ]:
print (test_df.isnull().sum())
test_df = test_df.fillna(0)
print (test_df.isnull().sum())

In [ ]:
sample_prediction_df

Note that we'll get an error if we try to continue on to the next batch without making our predictions for the current batch.

In [ ]:
next(iter_test)

### **`predict`** function
Stores your predictions for the current batch.  Expects the same format as you saw in `sample_prediction_df` returned from the `iter_test` generator.

Args:
* `predictions_df`: DataFrame which must have the same format as `sample_prediction_df`.

This function will raise an Exception if not called after a successful iteration of the `iter_test` generator.

Let's make a dummy prediction using the sample provided by `iter_test`.

In [ ]:
env.predict(sample_prediction_df)

## Main Loop
Let's loop through all the remaining batches in the test set generator and make the default prediction for each.  The `iter_test` generator will simply stop returning values once you've reached the end.

When writing your own Notebooks, be sure to write robust code that makes as few assumptions about the `iter_test`/`predict` loop as possible.  For example, the test set contains question IDs that have not been previously observed in train.

You may assume that the structure of `sample_prediction_df` will not change in this competition.

**The lecture rows in `test_df` should not be submitted.**

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    test_df['prior_question_had_explanation'] = test_df['prior_question_had_explanation'].astype(float)
    #test_df['prior_question_had_explanation'] = lbl.transform(list(test_df[['prior_question_had_explanation']].values.ravel()))
    #X_test = test_df.drop(['prior_group_answers_correct', 'prior_group_responses'], axis=1)
    X_test = test_df[['content_id', 'prior_question_had_explanation']]
    X_test = X_test.fillna(0.5) 
    #for model in iowa_model:
    y_pred = iowa_model.predict(X_test)
    #if y_pred < 0.5:
     #   y_pred = 0.5
    #print (y_pred.dtype)
    #, num_iteration=model.best_iteration)
    #y_preds.append(y_pred)

    #y_preds = sum(y_preds) / len(y_preds)
    test_df['answered_correctly'] = y_pred
    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])

for (test_df, sample_prediction_df) in iter_test:
    
    #val_predictions = sum(val_predictions) / len(val_predictions)
    test_df['answered_correctly'] = 0.5
    print (env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']]))

## Restart the Notebook to run your code again
In order to combat cheating, you are only allowed to call `make_env` or iterate through `iter_test` once per Notebook run.  However, while you're iterating on your model it's reasonable to try something out, change the model a bit, and try it again.  Unfortunately, if you try to simply re-run the code, or even refresh the browser page, you'll still be running on the same Notebook execution session you had been running before, and the `riideducation` module will still throw errors.  To get around this, you need to explicitly restart your Notebook execution session, which you can do by **clicking "Run"->"Restart Session"** in the Notebook Editor's menu bar at the top.